# HMM Surface Contacts

In [3]:
pip install pomegranate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.4/90.4 kB 813.7 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.7/154.7 kB 5.8 MB/s eta 0:00:00
  Created wheel for apricot-select: filename=apricot_select-0.6.1-py3-none-any.whl size=48768 sha256=ed2d430ee2385469a9bcc32b79f12564d2ddc2b5a4e13e5f9bc31081225a0e3e
  Stored in directory: /root/.cache/pip/wheels/df/d8/f9/4d62b7272bff772a126a52e507212c2fd33c0b8416d9389665
Successfully built apricot-select


In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from sklearn.preprocessing import LabelEncoder
#from hmmlearn import hmm
from pomegranate import *
from sklearn.model_selection import train_test_split

In [ ]:
#pip install hmmlearn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.9/192.9 KB 4.0 MB/s eta 0:00:00a 0:00:01
You should consider upgrading via the '/opt/homebrew/Caskroom/miniforge/base/envs/tensorflow/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [6]:
# import data from data

df = pd.read_csv('movsdf.rbind_orientationcorrected.csv')
df.head()
print(len(df))

# remove any NAN values
df = df.dropna()

# filter only first 20 rows


1164


In [7]:

# Create a new column combining 'CareType' and 'HCWType'
df['HiddenState'] = df['CareType'] + "_" + df['HCWType']

# Group by 'ActivityID' and create sequences
sequences = df.groupby('ActivityID')['SurfaceCategories'].apply(list)
hidden_states = df.groupby('ActivityID')['HiddenState'].apply(list)


sequences

ActivityID
1     [HygieneArea, In, FarPatient, FarPatient, FarP...
2     [HygieneArea, In, FarPatient, FarPatient, Pati...
3     [In, Equipment, FarPatient, FarPatient, FarPat...
4     [In, FarPatient, FarPatient, FarPatient, FarPa...
5     [HygieneArea, In, FarPatient, FarPatient, Pati...
                            ...                        
58    [In, FarPatient, FarPatient, NearPatient, Equi...
59    [In, FarPatient, FarPatient, Equipment, Equipm...
60    [In, FarPatient, FarPatient, Equipment, Equipm...
61    [In, FarPatient, Equipment, Equipment, NearPat...
62    [In, FarPatient, FarPatient, FarPatient, Equip...
Name: SurfaceCategories, Length: 62, dtype: object

In [ ]:
# calcualte summary statistics for the length of the surface sequences and plot a violin plot using seaborn
# Input: surface sequences
# Output: violin plot of surface sequence lengths

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt


# calculate length of each sequence
surface['length'] = sequences['sequence'].str.len()

# plot violin plot
sns.violinplot(x='length', data=surface)
plt.xlabel('Length of surface sequence')
plt.ylabel('Density')
plt.title('Length of surface sequences')
plt.show()


KeyError: 'sequence'

In [ ]:
# Fit a LabelEncoder for the sequences and hidden states
sequence_encoder = LabelEncoder()
sequence_encoder.fit(np.hstack(sequences.values))

hidden_state_encoder = LabelEncoder()
hidden_state_encoder.fit(np.hstack(hidden_states.values))

# Transform the sequences and hidden states to integers
sequences = sequences.apply(sequence_encoder.transform)
hidden_states = hidden_states.apply(hidden_state_encoder.transform)

# Split the sequences into a training set and a test set
sequences_train, sequences_test, hidden_states_train, hidden_states_test = train_test_split(
    sequences, hidden_states, test_size=0.2, random_state=42)

print(len(sequences_train), len(sequences_test), len(hidden_states_train), len(hidden_states_test))

sequences_train=sequences_train.tolist()
hidden_states_train=hidden_states_train.tolist()
sequences_test=sequences_test.tolist()

# Create a Multinomial HMM
model = hmm.MultinomialHMM(n_components=len(hidden_state_encoder.classes_))

# Prepare the data for the HMM
X = np.concatenate(sequences_train)
lengths = list(map(len, sequences_train))

# Fit the HMM
model.fit(X.reshape(-1, 1), lengths=lengths)

model
print(model.startprob_)
print(model.transmat_)

MultinomialHMM has undergone major changes. The previous version was implementing a CategoricalHMM (a special case of MultinomialHMM). This new implementation follows the standard definition for a Multinomial distribution (e.g. as in https://en.wikipedia.org/wiki/Multinomial_distribution). See these issues for details:
https://github.com/hmmlearn/hmmlearn/issues/335
https://github.com/hmmlearn/hmmlearn/issues/340


20 6 20 6
[nan nan nan nan nan nan]
[[nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]]


In [ ]:
# Generate a new sequence of surface contacts
_, new_sequence = model.sample(10)  # Change 10 to the desired sequence length

# The new sequence is an array of integers, so we use the inverse_transform method to convert it back to surface names
new_sequence_surfaces = sequence_encoder.inverse_transform(new_sequence.reshape(-1))

print(new_sequence_surfaces)


ValueError: startprob_ must sum to 1 (got nan)

In [ ]:
# Generate new sequences with the same lengths as the sequences in the test data
new_sequences = [model.sample(len(seq))[1] for seq in sequences_test]

# Calculate the lengths of the new sequences
new_lengths = [len(seq) for seq in new_sequences]

# Calculate the lengths of the sequences in the test data
test_lengths = [len(seq) for seq in sequences_test]

# Calculate the difference between the new lengths and the test lengths
length_difference = np.array(new_lengths) - np.array(test_lengths)

print(length_difference)


ValueError: startprob_ must sum to 1 (got nan)

## Pomegranate

In [ ]:
sequences.values

NameError: name 'sequences' is not defined

In [10]:
from pomegranate import HiddenMarkovModel, DiscreteDistribution
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Fit a LabelEncoder for the sequences and hidden states
sequence_encoder = LabelEncoder()
sequence_encoder.fit(np.hstack(sequences.values))

hidden_state_encoder = LabelEncoder()
hidden_state_encoder.fit(np.hstack(hidden_states.values))

# Transform the sequences and hidden states to integers
sequences = sequences.apply(sequence_encoder.transform)
hidden_states = hidden_states.apply(hidden_state_encoder.transform)

# Split the sequences into a training set and a test set
sequences_train, sequences_test, hidden_states_train, hidden_states_test = train_test_split(
    sequences, hidden_states, test_size=0.1, random_state=42)

sequences_train = sequences_train.tolist()
hidden_states_train = hidden_states_train.tolist()

# Prepare the data for the HMM
X = np.concatenate(sequences_train).reshape(-1, 1)
lengths = list(map(len, sequences_train))

# Create and train a Multivariate HMM
model = HiddenMarkovModel.from_samples(DiscreteDistribution,
                                       n_components=len(hidden_state_encoder.classes_),
                                       X=X,
                                       labels=hidden_states_train,
                                       algorithm='baum-welch',
                                       verbose=False,
                                       state_names=hidden_state_encoder.classes_.tolist(),
                                       n_jobs=-1)

# Now you can predict a new sequence
new_sequence = model.sample(length=10)  # Change 10 to the desired sequence length

# The new sequence is an array of integers, so we use the inverse_transform method to convert it back to surface names
new_sequence_surfaces = sequence_encoder.inverse_transform(new_sequence)

print(new_sequence_surfaces)


ImportError: ignored

In [ ]:
import sys
print(sys.path)


['/Users/marcofking/Documents/GitHub/MockVsActualHospitalCare/code', '/opt/homebrew/Caskroom/miniforge/base/envs/tensorflow/lib/python39.zip', '/opt/homebrew/Caskroom/miniforge/base/envs/tensorflow/lib/python3.9', '/opt/homebrew/Caskroom/miniforge/base/envs/tensorflow/lib/python3.9/lib-dynload', '', '/opt/homebrew/Caskroom/miniforge/base/envs/tensorflow/lib/python3.9/site-packages']
